In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')
!pip install conllu
!pip install SentencePiece
!pip install transformers[torch]
import utils
import models

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import ParameterGrid
import time
from collections import Counter

def evaluate_model(params):
    # Paramètres
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    optimizer_choice = params['optimizer']

    # Chargement des données d'entraînement et de validation
    sentences, pos_tags = utils.load_data("/content/gdrive/MyDrive/Colab Notebooks/UD_French-Sequoia/fr_sequoia-ud-train.conllu", pos=True)

    #ici je combien train et val pour entrainer une derniere fois à la fin (cest tres tres moche mais jai la flemme il est 3h du mat)
    #Pour executer bien comme il faut apres il faudra juste mettre la patience a haute valeur et mettre les epochs qu'on veuts comme ça cest bon (encore tres moche mais voila)

    # sentences2, pos_tags2 = utils.load_data("/content/gdrive/MyDrive/Colab Notebooks/UD_French-Sequoia/fr_sequoia-ud-dev.conllu", pos=True)
    # sentences = sentences + sentences2
    # pos_tags = pos_tags + pos_tags2


    val_sentences, val_pos_tags = utils.load_data("/content/gdrive/MyDrive/Colab Notebooks/UD_French-Sequoia/fr_sequoia-ud-dev.conllu", pos=True)
    limit = 10000
    sentences = sentences[:limit]
    pos_tags = pos_tags[:limit]
    val_sentences = val_sentences[:limit]
    val_pos_tags = val_pos_tags[:limit]

    # Dictionnaire des tags
    tag_counts = Counter(tag for tags in pos_tags for tag in tags)
    tag_to_ix = {tag: i+1 for i, tag in enumerate(tag_counts)}
    tag_to_ix['PAD'] = 0

    # DataLoaders pour l'entraînement et la validation
    dataset = models.BertPOSDataset(sentences, pos_tags, tag_to_ix)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=models.bert_collate_fn)
    val_dataset = models.BertPOSDataset(val_sentences, val_pos_tags, tag_to_ix)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=models.bert_collate_fn)

    # Modèle, perte et optimiseur
    model = models.BertPOSTagger(len(tag_to_ix)).to(utils.device)
    loss_function = nn.CrossEntropyLoss(ignore_index=tag_to_ix['PAD'])
    if optimizer_choice == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_choice == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Nombre maximal d'époques pour l'entraînement
    max_epochs = 15  # Modifiable selon vos besoins

    # Entraînement avec validation et arrêt anticipé
    train_start_time = time.time()
    train_loss, train_accuracy, valid_loss, valid_accuracy, epochs_used = utils.train2(model, data_loader, val_loader, loss_function, optimizer, tag_to_ix, max_epochs, early_stopping_rounds=100)
    train_time = time.time() - train_start_time
    # Évaluation sur le test set
    test_sentences, test_pos_tags = utils.load_data("/content/gdrive/MyDrive/Colab Notebooks/UD_French-Sequoia/fr_sequoia-ud-test.conllu", pos=True)
    test_dataset = models.BertPOSDataset(test_sentences, test_pos_tags, tag_to_ix)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=models.bert_collate_fn)
    test_start_time = time.time()
    test_loss, test_accuracy = utils.test_performance(model, test_loader, loss_function, tag_to_ix)
    test_time = time.time() - test_start_time

    return train_loss, train_accuracy, test_loss, test_accuracy, train_time, test_time, epochs_used

# Grille de recherche
param_grid = {
    'batch_size': [64],
    'learning_rate': [0.0001],
                      #,0.0002,0.0003,0.0004],
    'optimizer': ['Adam']
    #, 'SGD']
}

# Recherche des meilleurs hyperparamètres
best_accuracy = 0
best_params = None
best_results = None

for params in ParameterGrid(param_grid):
    train_loss, train_accuracy, test_loss, test_accuracy, train_time, test_time, epochs_used = evaluate_model(params)
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_params = params
        best_results = (train_loss, train_accuracy, test_loss, test_accuracy, train_time, test_time, epochs_used)

# Affichage des résultats avec le nombre d'époques utilisées
print(f"Meilleurs paramètres : {best_params}")
print(f"Nombre d'époques utilisées : {best_results[-1]}")
print(f"Meilleure précision sur le train : {best_results[1]}, sur le test : {best_results[3]}")
print(f"Loss sur le train : {best_results[0]}, sur le test : {best_results[2]}")
print(f"Temps d'entraînement : {best_results[4]}, Temps d'évaluation : {best_results[5]}")


Epoch 1, Train Loss: 0.902943247130939, Valid Loss: 0.31781552944864544, Train Accuracy: 0.7372292603055636, Valid Accuracy: 0.9023941499846322
Time elapsed: 0.8952654957771301 min
Epoch 2, Train Loss: 0.2620953342744282, Valid Loss: 0.15165258518287114, Train Accuracy: 0.9195292918455034, Valid Accuracy: 0.9541442138808114
Time elapsed: 1.769621527194977 min
Epoch 3, Train Loss: 0.16456077105942227, Valid Loss: 0.0912833714059421, Train Accuracy: 0.9491937117917197, Valid Accuracy: 0.9721015180860247
Time elapsed: 2.6542076468467712 min
Epoch 4, Train Loss: 0.12298983903158278, Valid Loss: 0.06527699530124664, Train Accuracy: 0.9620928040572575, Valid Accuracy: 0.9789069805826459
Time elapsed: 3.5361370205879212 min
Epoch 5, Train Loss: 0.08739593234800157, Valid Loss: 0.037804902132068365, Train Accuracy: 0.9721863113698506, Valid Accuracy: 0.9890471185956683
Time elapsed: 4.41866178115209 min
Epoch 6, Train Loss: 0.061582585087134725, Valid Loss: 0.029587479707385813, Train Accuracy